稳定扩散模型是在2022年引入的,它是开源的，预训练的权重可以在网上直接搜到。\
稳定扩散模型最简单的应用之一就是文本到图像的转换。也就是你给出一段指令来生成一张符合指令的图片。\
我们也将学习图片到图片的过程，它可以修改或者修复原先的图片。


在学习扩散模型之前，我们现需要知道什么是生成模型。\
生成模型就是先学习训练样本的生成分布,然后就可以通过采样来生成未曾见过但是类似的图片。\



**一个完整的模型包括两个过程**

前向过程（加噪声,有具体的解析公式），直到变成完全的噪声\
逆向过程是用来去除噪声的，没有具体公式，所以需要学习




完整的数学公式我将不在此赘述，只会选取重要的讲解，感兴趣的读者可以参考原文:\
Denoising Diffusion Probabilistic Models (DDPM)\
这篇文章并不难，还没有涉及到随机微分方程的部分，相信大家自行可以读懂，网上其他的解读也不甚繁多了。


**前向过程**

单步： $q(x_t|x_{t-1}) = \mathcal{N}\!\left(x_t; \sqrt{1-\beta_t}\,x_{t-1}, \beta_t\mathbf{I}\right)$ 

定义 $\alpha_t = 1-\beta_t$, $\bar{\alpha}_t = \prod_{s=1}^t \alpha_s$ 

任意步： $q(x_t|x_0) = \mathcal{N}\!\left(x_t; \sqrt{\bar{\alpha}_t}\,x_0, (1-\bar{\alpha}_t)\mathbf{I}\right)$ 

重参数化： $x_t = \sqrt{\bar{\alpha}_t} \, x_0 + \sqrt{1-\bar{\alpha}_t} \, \epsilon$,  $\epsilon \sim \mathcal{N}(0, \mathbf{I})$

这被称作为一个Markov过程

**反向过程**

我们需要让网络去学习去噪的参数。

**训练过程**

对多元概率做边缘化积分在计算上是不可行的，采用ELBO下界来计算。最后可以转化为对噪声进行最优化对的过程。

下面是原文的训练过程，是比较初级的版本

1. 重复
2. 从真实数据分布中采样：  
   $\mathbf{x}_0 \sim q(\mathbf{x}_0)$

3. 均匀采样时间步：  
   $t \sim \text{Uniform}(\{1, \ldots, T\})$

4. 采样高斯噪声：  
   $\boldsymbol{\epsilon} \sim \mathcal{N}(\mathbf{0}, \mathbf{I})$

5. 在以下损失函数上执行梯度下降：  
   $$
   \nabla_{\boldsymbol{\theta}} \left\| \boldsymbol{\epsilon} - \boldsymbol{\epsilon}_{\boldsymbol{\theta}} \left( \sqrt{\bar{\alpha}_t} \mathbf{x}_0 + \sqrt{1 - \bar{\alpha}_t} \boldsymbol{\epsilon}, t \right) \right\|^2
   $$
6. 直到收敛

得到这个网络以后，我们就可以从一个纯噪声出发去生成一个新的数据点（图片）。这便是无条件生成的所有步骤了，但是我们没法控制生成的方向，所以我们需要在去噪的每一步去加入提示（prompt）。

通过一些技术性的手段，可以得到一个好的结果。举个例子，比如说你想生成一张猫的图片，可以从一张加噪的猫开始，并给出提示，模型会去噪。在第二步中我们不给提示。这就是称为分类器引导。也就是在训练的时候，有时给出提示有时又不给提示。这样我们再添加权重，就可以控制输出有多接近我们的提示词。

下面讲一下神经网络中非常重要的NLP手段Clip。Clip是Openai构建的模型，它能够将文本与图像连接起来。它通过将文本与图像经过嵌入后做点积得到一个矩阵，对角元上是对应的，训练的时候就让对角元最大，其他接近0，通过这种方式们就能让文本和图像结合起来。在我们的稳定扩散模型中，使用的就是这个Clip作为文本编辑器，来得到我们的条件信号。

现在还需要考虑一下速度的问题，因为图片的处理非常慢，我们可以通过一个叫做变分自编码器的东西来压缩图像。事实上稳定扩散模型是一个潜在扩散模型，它学习的并不是数据的概率分布，而是再用变分自编码器学习数据的潜在表示，这就压缩了我们的数据。

然后我们具体看一下Autoencoder是怎么工作的，